In [3]:
import os
import sys
import json
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.optim import lr_scheduler

from opts import parse_opts
from model import generate_model
from mean import get_mean, get_std
from spatial_transforms import (
    Compose, Normalize, Scale, CenterCrop, CornerCrop, MultiScaleCornerCrop,
    MultiScaleRandomCrop, RandomHorizontalFlip, ToTensor)
from temporal_transforms import LoopPadding, TemporalRandomCrop
from target_transforms import ClassLabel, VideoID
from target_transforms import Compose as TargetCompose
from dataset import get_training_set, get_validation_set, get_test_set
from utils import Logger
from train import train_epoch
from validation import val_epoch
import test
import collections
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from joblib import dump, load
from sklearn import preprocessing
from scipy import stats
from sklearn.metrics import accuracy_score

from models import *

In [4]:
os.environ['CUDA_VISIBLE_DEVICES']='4,5'

In [5]:
from models.resnext import get_fine_tuning_parameters
from models import resnext

# construct model architecture
model_name = 'resnext-101-kinetics'
model = resnext.resnet101(
            num_classes=101,
            shortcut_type='B',
            cardinality=32,
            sample_size=112,
            sample_duration=16)

/home/qq/CSCE689/3D-ResNets-PyTorch/models/resnext.py:121: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


In [6]:
# load pretrained weight
model = model.cuda()
# wrap the current model again in nn.DataParallel / or we can just remove the .module keys.
model = nn.DataParallel(model, device_ids=None)
# filter out unnecessary keys
pretrain = torch.load('./pretrain/'+model_name+'.pth')
pretrain_dict = pretrain['state_dict']
# pretrain_dict_filter = {k: v for k, v in pretrain_dict.items() if k in model_dict}

pretrain_dict.pop('module.fc.weight')
pretrain_dict.pop('module.fc.bias')
model_dict = model.state_dict()
model_dict.update(pretrain_dict) 
model.load_state_dict(model_dict)

<All keys matched successfully>

In [ ]:
# summarize model input and output
from torchsummary import summary
summary(model, (3, 16, 112, 112))

In [7]:
# load preprocessed video frames and annotation
from datasets.ucf101 import UCF101
root_path = '/data/qq/CSCE689/video/'
video_path = root_path + 'UCF-101-jpg/' # 'UCF-music/' 
annotation_path = root_path+'ucfTrainTestlist/ucf101_01.json'

In [8]:
sample_size = 112 # res3d
sample_duration = 16 # for res3d
norm_value = 1
mean = get_mean(norm_value, dataset='kinetics')
std = get_std(norm_value)
norm_method = Normalize(mean, [1,1,1])

batch_size = 32
n_threads = 6

spatial_transform = Compose([
    Scale(sample_size),
    CornerCrop(sample_size, 'c'),
    ToTensor(norm_value), norm_method
])

temporal_transform = LoopPadding(sample_duration)
target_transform = ClassLabel() # VideoID()

# get training data
training_data = UCF101(
    video_path,
    annotation_path,
    'training',
    0,
    spatial_transform=spatial_transform,
    temporal_transform=temporal_transform,
    target_transform=target_transform,
    sample_duration=16)

# wrap training data
train_loader = torch.utils.data.DataLoader(
    training_data,
    batch_size=batch_size,
    shuffle=False,
    num_workers=n_threads,
    pin_memory=False) # True

# get validation data
val_data = UCF101(
    video_path,
    annotation_path,
    'validation',
    0,
    spatial_transform=spatial_transform,
    temporal_transform=temporal_transform,
    target_transform=target_transform,
    sample_duration=16)

# wrap validation data
val_loader = torch.utils.data.DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=False,
    num_workers=n_threads,
    pin_memory=False) # True



dataset loading [0/9537]
dataset loading [1000/9537]
dataset loading [2000/9537]
dataset loading [3000/9537]
dataset loading [4000/9537]
dataset loading [5000/9537]
dataset loading [6000/9537]
dataset loading [7000/9537]
dataset loading [8000/9537]
dataset loading [9000/9537]
dataset loading [0/3783]
dataset loading [1000/3783]
dataset loading [2000/3783]
dataset loading [3000/3783]


In [11]:
# set hyperparameters 

import easydict
opt = easydict.EasyDict({
    "root_path": '',
    "video_path": 'video_kinetics_jpg',
    "annotation_path": 'kinetics.json',
    "result_path": 'results1',
    "dataset": 'ucf101-music', # 'ucf101',
    "n_classes": 101, # 9, 
    "n_finetune_classes": 101, # 9,
    "sample_size": 112,
    "sample_duration": 16,
    "initial_scale": 1.0,
    "n_scales": 5,
    "scale_step": 0.84089641525,
    "train_crop": 'corner',
    "learning_rate": 0.1,
    "momentum": 0.9,
    "dampening": 0.9,
    "weight_decay": 0.001,
    "mean_dataset": 'activitynet',
    "no_mean_norm": False,
    "std_norm": False,
    "nesterov": False,
    "optimizer": 'sgd',
    "lr_patience": 10,
    "batch_size": 32,
    "n_epochs": 20,
    "begin_epoch": 1,
    "n_val_samples": 3,
    "resume_path": '',
    "pretrain_path": '',
    "ft_begin_index": 0,
    "no_train": False,
    "no_val": False,
    "test": False,
    "test_subset": 'val',
    "scale_in_test": 1.0,
    "crop_position_in_test": 'c',
    "no_softmax_in_test": False,
    "no_cuda": False,
    "n_threads": 4,
    "checkpoint": 10,
    "no_hflip": False,
    "norm_value": 1,
    "model": 'resnet',
    "model_depth": 18,
    "resnet_shortcut": 'B',
    "wide_resnet_k": 2,
    "resnext_cardinality": 32,
    "manual_seed": 1
})

In [12]:
criterion = nn.CrossEntropyLoss()
if not opt.no_cuda:
    criterion = criterion.cuda()
    
    
from models.resnext import get_fine_tuning_parameters

# get fine-tune parameters (we fine-tune all of them)
# parameters = model.parameters()
parameters = get_fine_tuning_parameters(model, opt.ft_begin_index)

optimizer = optim.SGD(
            parameters,
            lr=opt.learning_rate,
            momentum=opt.momentum,
            dampening=opt.dampening,
            weight_decay=opt.weight_decay,
            nesterov=opt.nesterov)

scheduler = lr_scheduler.ReduceLROnPlateau(
            optimizer, 'min', patience=opt.lr_patience)

train_logger = Logger(
    os.path.join(opt.result_path, 'train.log'),
    ['epoch', 'loss', 'acc', 'lr'])
train_batch_logger = Logger(
    os.path.join(opt.result_path, 'train_batch.log'),
    ['epoch', 'batch', 'iter', 'loss', 'acc', 'lr'])
val_logger = Logger(
            os.path.join(opt.result_path, 'val.log'), ['epoch', 'loss', 'acc'])

for i in range(opt.begin_epoch, opt.n_epochs + 1):

    train_epoch(i, train_loader, model, criterion, optimizer, 
                opt, train_logger, train_batch_logger)
    
    validation_loss = val_epoch(i, val_loader, model, criterion, 
                                opt, val_logger)
    
    scheduler.step(validation_loss)

train at epoch 1
Epoch: [1][1/3611]	Time 2.434 (2.434)	Data 1.800 (1.800)	Loss 4.8661 (4.8661)	Acc 0.000 (0.000)
Epoch: [1][2/3611]	Time 0.589 (1.512)	Data 0.131 (0.966)	Loss 7.3180 (6.0921)	Acc 0.000 (0.000)
Epoch: [1][3/3611]	Time 0.471 (1.165)	Data 0.013 (0.648)	Loss 15.0944 (9.0928)	Acc 0.000 (0.000)
Epoch: [1][4/3611]	Time 0.474 (0.992)	Data 0.014 (0.490)	Loss 19.9465 (11.8062)	Acc 0.000 (0.000)
Epoch: [1][5/3611]	Time 0.479 (0.889)	Data 0.013 (0.394)	Loss 12.3341 (11.9118)	Acc 0.000 (0.000)
Epoch: [1][6/3611]	Time 0.472 (0.820)	Data 0.014 (0.331)	Loss 20.3010 (13.3100)	Acc 0.000 (0.000)
Epoch: [1][7/3611]	Time 0.499 (0.774)	Data 0.028 (0.287)	Loss 16.4721 (13.7617)	Acc 0.281 (0.040)
Epoch: [1][8/3611]	Time 0.471 (0.736)	Data 0.018 (0.254)	Loss 21.0361 (14.6710)	Acc 0.000 (0.035)
Epoch: [1][9/3611]	Time 0.491 (0.709)	Data 0.013 (0.227)	Loss 21.4514 (15.4244)	Acc 0.000 (0.031)
Epoch: [1][10/3611]	Time 0.467 (0.685)	Data 0.013 (0.206)	Loss 20.2373 (15.9057)	Acc 0.000 (0.028)
Epoch: 

Epoch: [1][85/3611]	Time 0.479 (0.504)	Data 0.016 (0.041)	Loss 4.7037 (7.1182)	Acc 0.000 (0.014)
Epoch: [1][86/3611]	Time 0.484 (0.503)	Data 0.023 (0.041)	Loss 4.4817 (7.0876)	Acc 0.000 (0.014)
Epoch: [1][87/3611]	Time 0.484 (0.503)	Data 0.021 (0.041)	Loss 4.8402 (7.0617)	Acc 0.000 (0.014)
Epoch: [1][88/3611]	Time 0.471 (0.503)	Data 0.015 (0.040)	Loss 4.9843 (7.0381)	Acc 0.000 (0.013)
Epoch: [1][89/3611]	Time 0.480 (0.503)	Data 0.016 (0.040)	Loss 4.1592 (7.0058)	Acc 0.000 (0.013)
Epoch: [1][90/3611]	Time 0.489 (0.502)	Data 0.018 (0.040)	Loss 4.5876 (6.9789)	Acc 0.000 (0.013)
Epoch: [1][91/3611]	Time 0.492 (0.502)	Data 0.023 (0.040)	Loss 4.9959 (6.9571)	Acc 0.000 (0.013)
Epoch: [1][92/3611]	Time 0.487 (0.502)	Data 0.023 (0.039)	Loss 4.3344 (6.9286)	Acc 0.000 (0.013)
Epoch: [1][93/3611]	Time 0.479 (0.502)	Data 0.015 (0.039)	Loss 4.4262 (6.9017)	Acc 0.000 (0.013)
Epoch: [1][94/3611]	Time 0.483 (0.502)	Data 0.024 (0.039)	Loss 4.8138 (6.8795)	Acc 0.000 (0.013)
Epoch: [1][95/3611]	Time 0.494

Epoch: [1][169/3611]	Time 0.472 (0.496)	Data 0.016 (0.030)	Loss 4.7966 (5.8799)	Acc 0.000 (0.010)
Epoch: [1][170/3611]	Time 0.482 (0.496)	Data 0.017 (0.030)	Loss 4.5664 (5.8722)	Acc 0.000 (0.010)
Epoch: [1][171/3611]	Time 0.487 (0.496)	Data 0.022 (0.030)	Loss 4.5682 (5.8645)	Acc 0.000 (0.010)
Epoch: [1][172/3611]	Time 0.483 (0.496)	Data 0.023 (0.030)	Loss 4.5939 (5.8571)	Acc 0.000 (0.010)
Epoch: [1][173/3611]	Time 0.485 (0.496)	Data 0.023 (0.030)	Loss 4.5653 (5.8497)	Acc 0.000 (0.010)
Epoch: [1][174/3611]	Time 0.482 (0.496)	Data 0.018 (0.030)	Loss 4.6964 (5.8431)	Acc 0.000 (0.010)
Epoch: [1][175/3611]	Time 0.498 (0.496)	Data 0.024 (0.030)	Loss 4.9450 (5.8379)	Acc 0.000 (0.010)
Epoch: [1][176/3611]	Time 0.479 (0.496)	Data 0.017 (0.030)	Loss 4.4365 (5.8300)	Acc 0.000 (0.010)
Epoch: [1][177/3611]	Time 0.485 (0.496)	Data 0.024 (0.030)	Loss 4.3839 (5.8218)	Acc 0.000 (0.010)
Epoch: [1][178/3611]	Time 0.467 (0.496)	Data 0.016 (0.030)	Loss 4.7667 (5.8159)	Acc 0.000 (0.009)
Epoch: [1][179/3611]

Epoch: [1][253/3611]	Time 0.487 (0.493)	Data 0.015 (0.027)	Loss 4.8370 (5.4446)	Acc 0.000 (0.012)
Epoch: [1][254/3611]	Time 0.481 (0.493)	Data 0.016 (0.027)	Loss 5.0118 (5.4429)	Acc 0.000 (0.012)
Epoch: [1][255/3611]	Time 0.481 (0.493)	Data 0.016 (0.027)	Loss 4.3805 (5.4387)	Acc 0.000 (0.012)
Epoch: [1][256/3611]	Time 0.498 (0.493)	Data 0.023 (0.027)	Loss 4.8282 (5.4363)	Acc 0.000 (0.012)
Epoch: [1][257/3611]	Time 0.479 (0.493)	Data 0.022 (0.027)	Loss 5.0679 (5.4349)	Acc 0.000 (0.012)
Epoch: [1][258/3611]	Time 0.492 (0.493)	Data 0.018 (0.026)	Loss 4.8140 (5.4325)	Acc 0.000 (0.012)
Epoch: [1][259/3611]	Time 0.485 (0.493)	Data 0.017 (0.026)	Loss 4.2203 (5.4278)	Acc 0.156 (0.013)
Epoch: [1][260/3611]	Time 0.482 (0.493)	Data 0.017 (0.026)	Loss 4.8227 (5.4255)	Acc 0.000 (0.013)
Epoch: [1][261/3611]	Time 0.488 (0.493)	Data 0.022 (0.026)	Loss 4.5918 (5.4223)	Acc 0.000 (0.012)
Epoch: [1][262/3611]	Time 0.489 (0.493)	Data 0.016 (0.026)	Loss 4.1066 (5.4173)	Acc 0.188 (0.013)
Epoch: [1][263/3611]

Epoch: [1][337/3611]	Time 0.496 (0.492)	Data 0.024 (0.025)	Loss 4.9225 (5.2343)	Acc 0.000 (0.017)
Epoch: [1][338/3611]	Time 0.481 (0.492)	Data 0.017 (0.025)	Loss 4.7496 (5.2329)	Acc 0.000 (0.017)
Epoch: [1][339/3611]	Time 0.491 (0.492)	Data 0.024 (0.025)	Loss 4.6178 (5.2310)	Acc 0.000 (0.017)
Epoch: [1][340/3611]	Time 0.482 (0.492)	Data 0.018 (0.025)	Loss 4.7394 (5.2296)	Acc 0.000 (0.017)
Epoch: [1][341/3611]	Time 0.483 (0.492)	Data 0.018 (0.025)	Loss 3.5325 (5.2246)	Acc 0.781 (0.019)
Epoch: [1][342/3611]	Time 0.492 (0.492)	Data 0.018 (0.025)	Loss 4.3132 (5.2220)	Acc 0.000 (0.019)
Epoch: [1][343/3611]	Time 0.510 (0.492)	Data 0.022 (0.025)	Loss 4.6594 (5.2203)	Acc 0.000 (0.019)
Epoch: [1][344/3611]	Time 0.484 (0.492)	Data 0.016 (0.025)	Loss 4.0405 (5.2169)	Acc 0.344 (0.020)
Epoch: [1][345/3611]	Time 0.487 (0.492)	Data 0.016 (0.025)	Loss 4.6036 (5.2151)	Acc 0.094 (0.020)
Epoch: [1][346/3611]	Time 0.493 (0.492)	Data 0.019 (0.024)	Loss 4.5850 (5.2133)	Acc 0.000 (0.020)
Epoch: [1][347/3611]

...

Epoch: [1][1009/3611]	Time 0.479 (0.488)	Data 0.017 (0.020)	Loss 4.6420 (4.7916)	Acc 0.000 (0.024)
Epoch: [1][1010/3611]	Time 0.477 (0.488)	Data 0.017 (0.020)	Loss 3.7976 (4.7906)	Acc 0.000 (0.024)
Epoch: [1][1011/3611]	Time 0.477 (0.488)	Data 0.016 (0.020)	Loss 4.6930 (4.7905)	Acc 0.000 (0.024)
Epoch: [1][1012/3611]	Time 0.479 (0.488)	Data 0.016 (0.020)	Loss 4.2977 (4.7900)	Acc 0.000 (0.024)
Epoch: [1][1013/3611]	Time 0.474 (0.488)	Data 0.014 (0.020)	Loss 4.5877 (4.7898)	Acc 0.000 (0.024)
Epoch: [1][1014/3611]	Time 0.493 (0.488)	Data 0.023 (0.020)	Loss 4.6698 (4.7897)	Acc 0.000 (0.024)
Epoch: [1][1015/3611]	Time 0.495 (0.488)	Data 0.016 (0.020)	Loss 4.5105 (4.7894)	Acc 0.000 (0.024)
Epoch: [1][1016/3611]	Time 0.485 (0.488)	Data 0.015 (0.020)	Loss 4.4340 (4.7891)	Acc 0.000 (0.024)
Epoch: [1][1017/3611]	Time 0.480 (0.488)	Data 0.014 (0.020)	Loss 4.6347 (4.7889)	Acc 0.000 (0.024)
Epoch: [1][1018/3611]	Time 0.483 (0.488)	Data 0.017 (0.020)	Loss 4.5441 (4.7887)	Acc 0.000 (0.024)
Epoch: [1]

Epoch: [1][1092/3611]	Time 0.498 (0.487)	Data 0.016 (0.020)	Loss 4.3166 (4.7707)	Acc 0.000 (0.024)
Epoch: [1][1093/3611]	Time 0.484 (0.487)	Data 0.016 (0.020)	Loss 4.4904 (4.7705)	Acc 0.000 (0.024)
Epoch: [1][1094/3611]	Time 0.478 (0.487)	Data 0.017 (0.020)	Loss 4.6650 (4.7704)	Acc 0.000 (0.024)
Epoch: [1][1095/3611]	Time 0.479 (0.487)	Data 0.014 (0.020)	Loss 4.3963 (4.7700)	Acc 0.000 (0.024)
Epoch: [1][1096/3611]	Time 0.485 (0.487)	Data 0.016 (0.020)	Loss 4.7316 (4.7700)	Acc 0.000 (0.024)
Epoch: [1][1097/3611]	Time 0.480 (0.487)	Data 0.017 (0.020)	Loss 4.7156 (4.7700)	Acc 0.000 (0.024)
Epoch: [1][1098/3611]	Time 0.479 (0.487)	Data 0.015 (0.020)	Loss 4.6006 (4.7698)	Acc 0.000 (0.024)
Epoch: [1][1099/3611]	Time 0.496 (0.487)	Data 0.025 (0.020)	Loss 5.0798 (4.7701)	Acc 0.000 (0.024)
Epoch: [1][1100/3611]	Time 0.477 (0.487)	Data 0.017 (0.020)	Loss 4.9424 (4.7702)	Acc 0.000 (0.024)
Epoch: [1][1101/3611]	Time 0.478 (0.487)	Data 0.016 (0.020)	Loss 4.9410 (4.7704)	Acc 0.000 (0.024)
Epoch: [1]

Epoch: [1][1175/3611]	Time 0.477 (0.487)	Data 0.017 (0.020)	Loss 4.5765 (4.7570)	Acc 0.000 (0.024)
Epoch: [1][1176/3611]	Time 0.485 (0.487)	Data 0.024 (0.020)	Loss 4.5448 (4.7568)	Acc 0.000 (0.024)
Epoch: [1][1177/3611]	Time 0.483 (0.487)	Data 0.015 (0.020)	Loss 4.7112 (4.7568)	Acc 0.000 (0.024)
Epoch: [1][1178/3611]	Time 0.476 (0.487)	Data 0.016 (0.020)	Loss 4.7014 (4.7567)	Acc 0.000 (0.024)
Epoch: [1][1179/3611]	Time 0.478 (0.487)	Data 0.014 (0.020)	Loss 4.3255 (4.7564)	Acc 0.000 (0.024)
Epoch: [1][1180/3611]	Time 0.478 (0.487)	Data 0.017 (0.020)	Loss 4.6897 (4.7563)	Acc 0.000 (0.024)
Epoch: [1][1181/3611]	Time 0.476 (0.487)	Data 0.015 (0.020)	Loss 4.7247 (4.7563)	Acc 0.000 (0.024)
Epoch: [1][1182/3611]	Time 0.478 (0.487)	Data 0.015 (0.020)	Loss 4.8144 (4.7563)	Acc 0.000 (0.024)
Epoch: [1][1183/3611]	Time 0.483 (0.487)	Data 0.019 (0.020)	Loss 4.6458 (4.7562)	Acc 0.000 (0.024)
Epoch: [1][1184/3611]	Time 0.478 (0.487)	Data 0.016 (0.020)	Loss 4.5256 (4.7560)	Acc 0.000 (0.024)
Epoch: [1]

Epoch: [1][1258/3611]	Time 0.483 (0.487)	Data 0.017 (0.019)	Loss 4.5963 (4.7453)	Acc 0.000 (0.023)
Epoch: [1][1259/3611]	Time 0.474 (0.487)	Data 0.015 (0.019)	Loss 4.8191 (4.7454)	Acc 0.000 (0.023)
Epoch: [1][1260/3611]	Time 0.476 (0.487)	Data 0.015 (0.019)	Loss 4.6618 (4.7453)	Acc 0.000 (0.023)
Epoch: [1][1261/3611]	Time 0.499 (0.487)	Data 0.024 (0.019)	Loss 4.8342 (4.7454)	Acc 0.000 (0.023)
Epoch: [1][1262/3611]	Time 0.478 (0.487)	Data 0.017 (0.019)	Loss 4.5850 (4.7453)	Acc 0.000 (0.023)
Epoch: [1][1263/3611]	Time 0.485 (0.487)	Data 0.021 (0.019)	Loss 4.7429 (4.7453)	Acc 0.000 (0.023)
Epoch: [1][1264/3611]	Time 0.484 (0.487)	Data 0.022 (0.019)	Loss 4.6508 (4.7452)	Acc 0.000 (0.023)
Epoch: [1][1265/3611]	Time 0.485 (0.487)	Data 0.023 (0.019)	Loss 4.6924 (4.7451)	Acc 0.000 (0.023)
Epoch: [1][1266/3611]	Time 0.490 (0.487)	Data 0.024 (0.019)	Loss 4.5315 (4.7450)	Acc 0.000 (0.023)
Epoch: [1][1267/3611]	Time 0.490 (0.487)	Data 0.025 (0.019)	Loss 4.2422 (4.7446)	Acc 0.062 (0.023)
Epoch: [1]

KeyboardInterrupt: 

In [35]:
root_path = '/data/qq/CSCE689/video/'
video_path = root_path + 'UCF-music/'  # 'UCF-101-jpg/' 
annotation_path = root_path+'ucfTrainTestlist/ucf101_01_music.json'

In [36]:
import json
with open(annotation_path, 'r') as data_file:
        qq1 = json.load(data_file)

In [37]:
len(qq1['database'])

1267

In [38]:
root_path = '/data/qq/CSCE689/video/'
video_path = root_path + 'UCF-music/'  # 'UCF-101-jpg/' 
annotation_path = root_path+'ucfTrainTestlist/ucf101_02_music.json'
import json
with open(annotation_path, 'r') as data_file:
        qq2 = json.load(data_file)

In [39]:
len(qq2['database'])

1267